In [2]:
import numpy as np
import pandas as pd
data = pd.read_json('/content/drive/MyDrive/Dissertation_review/merged/final_data-review.json')


Baseline model 1


In [3]:

data.head()

,id,conference,decision,url,hasContent,hasReview,title,authors,reviews,metaReview,name,metadata
0,ICLR_2017_381,ICLR,Reject,http://openreview.net/pdf/63b92f7ce8708e9df35d...,true,true,Multi-task learning with deep model based rein...,[Asier Mujika],[{'review': 'The term strategy is a bit ambigu...,None,ICLR_2017_381.pdf,"{'source': 'CRF', 'title': 'MULTI-TASK LEARNIN..."
1,ICLR_2017_211,ICLR,Invite to Workshop Track,http://openreview.net/pdf/5c54afd2ea867c72b8ae...,true,true,Nonparametrically Learning Activation Function...,"[Carson Eisenach, Zhaoran Wang, Han Liu]",[{'review': 'This paper provides a principled ...,None,ICLR_2017_211.pdf,"{'source': 'CRF', 'title': 'NONPARAMETRICALLY ..."
2,ICLR_2017_200,ICLR,Invite to Workshop Track,http://openreview.net/pdf/e2d348d16ad96d19425e...,true,true,Discovering objects and their relations from e...,"[David Raposo, Adam Santoro, David Barrett, Ra...",[{'review': '+ Understanding relations between...,None,ICLR_2017_200.pdf,"{'source': 'CRF', 'title': 'DISCOVERING OBJECT..."
3,ICLR_2017_106,ICLR,Accept (Poster),http://openreview.net/pdf/5ab63afda67c68cd39a6...,true,true,"Attend, Adapt and Transfer: Attentive Deep Arc...","[Janarthanan Rajendran, Aravind Lakshminarayan...",[{'review': 'In this paper a well known soft m...,None,ICLR_2017_106.pdf,"{'source': 'CRF', 'title': 'TRANSFER FROM MULT..."
4,ICLR_2017_64,ICLR,Accept (Poster),http://openreview.net/pdf/2cb01001f2f89ca11252...,true,true,Training Compressed Fully-Connected Networks w...,"[Shengjie Wang, Haoran Cai, Jeff Bilmes, Willi...",[{'review': 'The method proposes to compress t...,None,ICLR_2017_64.pdf,"{'source': 'CRF', 'title': None, 'authors': ['..."


In [25]:
has_none_string = data['metadata'].apply(lambda x: x == "None").any()
print(has_none_string)

False


In [26]:
data['metadata'].iloc[5]

{'source': 'CRF',
 'title': 'Transfer Learning with Neural AutoML',
 'authors': ['Catherine Wong', 'Neil Houlsby', 'Yifeng Lu'],
 'emails': ['catwong@mit.edu',
  'neilhoulsby@google.com',
  'yifenglu@google.com',
  'agesmundo@google.com'],
 'sections': [{'heading': None,
   'text': 'We reduce the computational cost of Neural AutoML with transfer learning. AutoML relieves human effort by automating the design of ML algorithms. Neural AutoML has become popular for the design of deep learning architectures, however, this method has a high computation cost. To address this we propose Transfer Neural AutoML that uses knowledge from prior tasks to speed up network design. We extend RL-based architecture search methods to support parallel training on multiple tasks and then transfer the search strategy to new tasks. On language and image classification tasks, Transfer Neural AutoML reduces convergence time over single-task training by over an order of magnitude on many tasks.'},
  {'heading':

In [23]:
data.isnull().sum()

,0
id,0
conference,0
decision,0
url,0
hasContent,0
hasReview,0
title,0
authors,0
reviews,0
metaReview,24


In [9]:

!pip install datasets
import nltk
nltk.download('punkt')
!pip install torch transformers bert-score
!pip install tqdm
!pip install rouge_score
!pip install sentence_transformers


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
import json
import os
from tqdm import tqdm
from typing import List, Dict
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import bert_score
from bert_score import score

def clean_text(text):
    if text is None:
        return ""
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def extract_introduction(metadata):
    """Extracting the introduction section if available. If not, returns None."""
    if metadata is not None and isinstance(metadata, dict):
        sections = metadata.get('sections', [])
        for section in sections:
            if section and section.get('heading'):  # Ensure section and heading are valid
                heading = section.get('heading', '').lower()
                if 'introduction' in heading:  # Look for "introduction"
                    text = section.get('text', '')
                    return clean_text(text) if text else None  # Return cleaned introduction text
    return None  # Return None if no valid introduction found



# TF-IDF based sentence extraction
class TFIDFBaselineExtractor:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(stop_words='english')

    def extract_salient_sentences(self, text: str, num_sentences: int = 3) -> str:
        if not text:
            return ''

        sentences = self._split_into_sentences(text)
        if len(sentences) <= num_sentences:
            return " ".join(sentences)

        tfidf_matrix = self.vectorizer.fit_transform(sentences)
        sentence_scores = np.sum(tfidf_matrix.toarray(), axis=1)
        top_sentence_indices = sentence_scores.argsort()[-num_sentences:][::-1]

        return " ".join([sentences[i] for i in sorted(top_sentence_indices)])

    def _split_into_sentences(self, text: str) -> list:
        return [s.strip() for s in text.split('.') if s.strip()]






In [32]:
extract_introduction(data['metadata'].iloc[5])

'automatic machine learning (automl) aims to find the best performing learning algorithms with minimal human intervention. many automl methods exist, including random search [1], performance modelling [2, 3], bayesian optimization [4], genetic algorithms [5, 6] and rl [7, 8]. we focus on neural automl, that uses deep rl to optimize architectures. these methods have shown promising results. for example, neural architecture search has discovered novel networks that rival the best human-designed architectures on challenging image classification tasks [9, 10]. however, neural automl is expensive because it requires training many networks. this may require vast computations resources; zoph and le [8] report 800 concurrent gpus to train on cifar-10. further, training needs to be repeated for every new task. some methods have been proposed to address this cost, such as using a progressive search space [11], or by sharing weights among generated networks [12, 13]. we propose a complementary so

In [11]:
# Generating review based on extracted introduction
def generate_review(metadata):
    introduction = extract_introduction(metadata)
    if introduction: # Check if introduction is not None
        extractor = TFIDFBaselineExtractor()
        return extractor.extract_salient_sentences(introduction, num_sentences=8)
    return ''  # Return empty if no introduction

# Function to calculate BERTScore for evaluation
def calculate_bertscore(introduction, generated_review):
    """
    This function calculates the BERTScore between the introduction and the generated review.
    """
    if not introduction or not generated_review:
        return None, None, None

    # Calculate BERTScore
    P, R, F1 = score([generated_review], [introduction], lang="en", verbose=True)

    return P.mean().item(), R.mean().item(), F1.mean().item()

# Process the dataset
def process_dataset(data: pd.DataFrame):
    processed_data = []

    # Iterate through the dataset rows
    for idx, row in tqdm(data.iterrows(), total=len(data)):
        metadata = row['metadata']
        try:
            generated_review = generate_review(metadata)
        except TypeError:  # Catch the TypeError that was causing the issue
            print(f"Skipping row {idx} due to error.")
            continue  # Move on to the next row

        if generated_review:  # Proceed only if there's a generated review
            introduction = extract_introduction(metadata)


            processed_data.append({
                'paper_id': row['id'],
                'title': row.get('title', 'unknown'),
                'metadata': metadata,
                'generated_review': generated_review,

            })

    # Convert to DataFrame for easy saving/processing
    return pd.DataFrame(processed_data)
processed_data = process_dataset(data)

 93%|█████████▎| 8027/8614 [01:02<00:02, 236.20it/s]

Skipping row 8000 due to error.


100%|██████████| 8614/8614 [01:05<00:00, 132.51it/s]


In [12]:
# Save the processed data
processed_data.to_csv('generated_reviews-baseline.csv', index=False)

# Example of how to access the first few results
print(processed_data.head())

        paper_id                                              title  \
0  ICLR_2017_381  Multi-task learning with deep model based rein...   
1  ICLR_2017_211  Nonparametrically Learning Activation Function...   
2  ICLR_2017_200  Discovering objects and their relations from e...   
3  ICLR_2017_106  Attend, Adapt and Transfer: Attentive Deep Arc...   
4   ICLR_2017_64  Training Compressed Fully-Connected Networks w...   

                                            metadata  \
0  {'source': 'CRF', 'title': 'MULTI-TASK LEARNIN...   
1  {'source': 'CRF', 'title': 'NONPARAMETRICALLY ...   
2  {'source': 'CRF', 'title': 'DISCOVERING OBJECT...   
3  {'source': 'CRF', 'title': 'TRANSFER FROM MULT...   
4  {'source': 'CRF', 'title': None, 'authors': ['...   

                                    generated_review  
0  recently, there has been a lot of success in a...  
1  deep learning techniques have proven particula...  
2  for example, object relations enable the trans...  
3  transferring 

In [22]:
processed_data['generated_review'].iloc[1]

'reinforcement learning (rl) is a field of research that uses dynamic programing (dp; bertsekas 2008), among other approaches, to solve sequential decision making problems the main challenge in applying dp to real world problems is an exponential growth of computational requirements as the problem size increases, known as the curse of dimensionality (bertsekas, 2008) popular function approximators for this task include deep neural networks, henceforth termed deep rl (drl), and linear architectures, henceforth termed shallow rl (srl) in particular, batch algorithms based on a least squares (ls) approach, such as least squares temporal difference (lstd, lagoudakis & parr 2003) and fitted-q iteration (fqi, ernst et al recent advancements in drl using convolutional neural networks demonstrated learning of expressive features (zahavy et al this motivates us to ask: can we combine drl with srl to leverage the benefits of both? in this work, we develop a hybrid approach that combines batch sr

In [15]:
from bert_score import score


introduction = processed_data.iloc[1]['introduction']
generated_review = " ".join(processed_data.iloc[1]['generated_review'])

# Calculate BERTScore
P, R, F1 = score([generated_review], [introduction], lang="en", verbose=True)

# Print the results
print(f"BERTScore Precision: {P.mean().item()}")
print(f"BERTScore Recall: {R.mean().item()}")
print(f"BERTScore F1: {F1.mean().item()}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.26 seconds, 3.82 sentences/sec
BERTScore Precision: 0.6023514270782471
BERTScore Recall: 0.7386294603347778
BERTScore F1: 0.6635658144950867


In [14]:
introduction = extract_introduction(processed_data.iloc[0]['metadata']) # Extract introduction from metadata
generated_review = " ".join(processed_data.iloc[0]['generated_review'])

# Calculate BERTScore
P, R, F1 = score([generated_review], [introduction], lang="en", verbose=True)

# Print the results
print(f"BERTScore Precision: {P.mean().item()}")
print(f"BERTScore Recall: {R.mean().item()}")
print(f"BERTScore F1: {F1.mean().item()}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 11.10 seconds, 0.09 sentences/sec
BERTScore Precision: 0.6273438930511475
BERTScore Recall: 0.7395058870315552
BERTScore F1: 0.678822934627533


**This is a basic model which extracts information from Introduction ref bold text- Yuan, W., Liu, P. and Neubig, G., 2022. Can we automate scientific reviewing?**